# Bank Customers Clustering Analysis

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import date, timedelta
#for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler

 ## Load data

In [2]:
dataset = pd.read_csv(r'bank_transactions.csv')
dataset.head()
#print(dataset)

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


## Data cleaning

In [3]:
all_types = dataset.dtypes
print(all_types)
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(dataset.isnull().sum())
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
dataset[dataset.isnull().any(axis=1)]

TransactionID               object
CustomerID                  object
CustomerDOB                 object
CustGender                  object
CustLocation                object
CustAccountBalance         float64
TransactionDate             object
TransactionTime              int64
TransactionAmount (INR)    float64
dtype: object
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
TransactionID                 0
CustomerID                    0
CustomerDOB                3397
CustGender                 1100
CustLocation                151
CustAccountBalance         2369
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
29,T30,C8736958,19/12/86,M,GRAM PANCHAYTH HYDERABAD,NaN,3/8/16,160636,2600.0
176,T177,C5698953,1/1/1800,NaN,NAVI MUMBAI,8512.28,12/8/16,155727,3449.0
222,T223,C9022645,NaN,M,BHIWANDI,1730.30,21/10/16,150814,4000.0
1271,T1272,C2314972,29/7/93,M,THANE,NaN,21/10/16,203227,53.0
1495,T1496,C2515550,NaN,M,NEW DELHI,384.07,21/10/16,193326,2800.0
...,...,...,...,...,...,...,...,...,...
1047762,T1047763,C3827041,NaN,M,PUNE,91.36,18/9/16,193122,284.0
1047919,T1047920,C2427054,NaN,M,PUNE,91.36,18/9/16,201925,365.0
1048048,T1048049,C1927213,25/4/80,M,SALEM,NaN,18/9/16,82422,2200.0
1048152,T1048153,C8138543,NaN,F,REIS MAGOS BARDEZ,2630.67,18/9/16,181307,252.0


In [4]:
dataset = dataset.dropna(subset = ['CustomerDOB','CustGender', 'CustLocation', 'CustAccountBalance'], how = 'any', inplace=False)
print(dataset) # 1048567-6953 rows 

print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(dataset.isnull().sum())  #check that there are no more null values

        TransactionID CustomerID CustomerDOB CustGender   CustLocation  \
0                  T1   C5841053     10/1/94          F     JAMSHEDPUR   
1                  T2   C2142763      4/4/57          M        JHAJJAR   
2                  T3   C4417068    26/11/96          F         MUMBAI   
3                  T4   C5342380     14/9/73          F         MUMBAI   
4                  T5   C9031234     24/3/88          F    NAVI MUMBAI   
...               ...        ...         ...        ...            ...   
1048562      T1048563   C8020229      8/4/90          M      NEW DELHI   
1048563      T1048564   C6459278     20/2/92          M         NASHIK   
1048564      T1048565   C6412354     18/5/89          M      HYDERABAD   
1048565      T1048566   C6420483     30/8/78          M  VISAKHAPATNAM   
1048566      T1048567   C8337524      5/3/84          M           PUNE   

         CustAccountBalance TransactionDate  TransactionTime  \
0                  17819.05          2/8/16    

In [5]:
#count values in the column 'CustomerDOB'. There is a default date of birth when the real age is missing (1/1/1800).
print(dataset['CustomerDOB'].value_counts())

#we remove rows which contain this value
dataset = dataset.drop(dataset[dataset['CustomerDOB'] == '1/1/1800'].index,axis = 0)  

dataset["CustomerDOB"] = pd.to_datetime(dataset["CustomerDOB"])
print(dataset['CustomerDOB'].dtype)

print(dataset) #985322 rows x 9 columns now


1/1/1800    56292
1/1/89        809
1/1/90        784
6/8/91        698
1/1/91        665
            ...  
2/12/51         1
20/3/52         1
26/9/47         1
4/10/41         1
24/10/44        1
Name: CustomerDOB, Length: 17233, dtype: int64
datetime64[ns]
        TransactionID CustomerID CustomerDOB CustGender   CustLocation  \
0                  T1   C5841053  1994-10-01          F     JAMSHEDPUR   
1                  T2   C2142763  2057-04-04          M        JHAJJAR   
2                  T3   C4417068  1996-11-26          F         MUMBAI   
3                  T4   C5342380  1973-09-14          F         MUMBAI   
4                  T5   C9031234  1988-03-24          F    NAVI MUMBAI   
...               ...        ...         ...        ...            ...   
1048562      T1048563   C8020229  1990-08-04          M      NEW DELHI   
1048563      T1048564   C6459278  1992-02-20          M         NASHIK   
1048564      T1048565   C6412354  1989-05-18          M      HYDERABAD   


In [6]:
#remove 100 years where date is greater than date of the transaction (2016 for all records) or the person is less than 10 years old

#today = datetime.today().strftime('%Y-%m-%d') get today's date in this format
#print(today)
#dataset['todayVector'] = pd.to_datetime(today)
#print(dataset['todayVector'])
  
dataset['TransactionDate'] = pd.to_datetime(dataset['TransactionDate']) #convert to datetime
filtered_data = dataset[dataset['TransactionDate'].dt.year!=2016]
print(filtered_data)  #to verify that there are no records in this new dataset (all transaction were made in 2016)

#dataset['CustomerDOB'].mask(dataset['CustomerDOB'].dt.strftime('%Y') >= dataset['todayVector'].dt.strftime('%Y'), dataset['CustomerDOB'] - pd.DateOffset(years = 100) , inplace=True )

#If person has less than 10 years there is a mistake: we have to subtract 100 years also in this case.
#dataset['CustomerDOB'].mask((dataset['CustomerDOB'] - today < 10, dataset['CustomerDOB'] - pd.DateOffset(years = 100) , inplace=True )


#remove 100 years where date is greater than date of the transaction
dataset['CustomerDOB'].mask(dataset['CustomerDOB'].dt.year >= dataset['TransactionDate'].dt.year, dataset['CustomerDOB'] - pd.DateOffset(years = 100) , inplace=True )



#dataset['CustomerDOB'].mask((dataset['CustomerDOB'] - dataset['todayVector'])/ np.timedelta64(1, 'Y') < '18', dataset['CustomerDOB'] - pd.DateOffset(years = 100) , inplace=True )


#remove 100 years if the person is less than 10 years old
dataset['CustomerDOB'].mask(dataset['TransactionDate'].dt.year - dataset['CustomerDOB'].dt.year < 10 , dataset['CustomerDOB'] - pd.DateOffset(years = 100) , inplace=True )

#df['end_date'] - df['start_date']) / np.timedelta64(1, 'D')

#dataset['CustomerAGE'] = dataset['CustomerDOB'].apply(lambda x: (today - x).days // 365)

#create a CustomerAGE column in the dataset
dataset['CustomerAGE'] = dataset['TransactionDate']- dataset['CustomerDOB']
print(dataset['CustomerAGE'].dtype) #timedelta64[ns]

#to get only the year (as an integer number)
dataset['CustomerAGE'] = dataset['CustomerAGE'].astype('timedelta64[Y]').astype('int')
dataset.head()


Empty DataFrame
Columns: [TransactionID, CustomerID, CustomerDOB, CustGender, CustLocation, CustAccountBalance, TransactionDate, TransactionTime, TransactionAmount (INR)]
Index: []
timedelta64[ns]


,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAGE
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0,21
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,141858,27999.0,58
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.0,19
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0,42
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5,27


In [7]:
filtered_data = dataset[(dataset['CustomerDOB'].dt.year == 2000)] 
print(filtered_data) #269 rows


filtered_data = dataset[(dataset['CustomerAGE'] < 16)] #to see how many people between 16 and 10 years are left (2000-2006)
print(filtered_data)

#we consider that this is the real age because it is unlikely that they are more than 110 years old

        TransactionID CustomerID CustomerDOB CustGender CustLocation  \
734              T735   C1537725  2000-01-13          M    NEW DELHI   
6966            T6967   C1911373  2000-11-04          M          PEN   
7677            T7678   C1441125  2000-09-15          M       MUMBAI   
8090            T8091   C5842760  2000-12-09          M  NAVI MUMBAI   
8947            T8948   C6927025  2000-04-10          F      CHENNAI   
...               ...        ...         ...        ...          ...   
1025174      T1025175   C7840585  2000-07-02          M         PUNE   
1028878      T1028879   C5315956  2000-11-16          F       MUMBAI   
1033054      T1033055   C5121063  2000-09-09          M         AGRA   
1035639      T1035640   C9022891  2000-09-18          M    NEW DELHI   
1039268      T1039269   C4841383  2000-09-04          M         PUNE   

         CustAccountBalance TransactionDate  TransactionTime  \
734                  854.89      2016-10-21           223056   
6966   

In [8]:
#dataset['TransactionTime'] = dataset['TransactionTime'].astype('string')
#from datetime import datetime as dt

#from "AABBCC" to "AA:BB:CC"
#dataset['TransactionTime'] = dataset['TransactionTime'].str.replace(r'(\w{2}(?!$))', r'\1:', regex=True)

#print(dataset['TransactionTime'])

In [9]:
#dataset["TransactionTime"] = pd.to_datetime(dataset["TransactionTime"],format= '%H:%M:%S' ).dt.time
#dataset["TransactionTime"] = pd.datetime(dataset["TransactionTime"])

In [10]:
dataset.drop('TransactionTime', inplace=True, axis=1)  #Unix format (It measures time by the number of seconds that have elapsed since 00:00:00 UTC on 1 January 1970)
print(dataset)

        TransactionID CustomerID CustomerDOB CustGender   CustLocation  \
0                  T1   C5841053  1994-10-01          F     JAMSHEDPUR   
1                  T2   C2142763  1957-04-04          M        JHAJJAR   
2                  T3   C4417068  1996-11-26          F         MUMBAI   
3                  T4   C5342380  1973-09-14          F         MUMBAI   
4                  T5   C9031234  1988-03-24          F    NAVI MUMBAI   
...               ...        ...         ...        ...            ...   
1048562      T1048563   C8020229  1990-08-04          M      NEW DELHI   
1048563      T1048564   C6459278  1992-02-20          M         NASHIK   
1048564      T1048565   C6412354  1989-05-18          M      HYDERABAD   
1048565      T1048566   C6420483  1978-08-30          M  VISAKHAPATNAM   
1048566      T1048567   C8337524  1984-05-03          M           PUNE   

         CustAccountBalance TransactionDate  TransactionAmount (INR)  \
0                  17819.05      2016-0

In [11]:
all_types = dataset.dtypes
print(all_types) 
dataset.head()

TransactionID                      object
CustomerID                         object
CustomerDOB                datetime64[ns]
CustGender                         object
CustLocation                       object
CustAccountBalance                float64
TransactionDate            datetime64[ns]
TransactionAmount (INR)           float64
CustomerAGE                         int32
dtype: object


,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionAmount (INR),CustomerAGE
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,25.0,21
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,27999.0,58
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,459.0,19
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,2060.0,42
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,1762.5,27


In [12]:
dataset['TransactionID'] = dataset['TransactionID'].replace({'T' : ''}, regex=True)
dataset['TransactionID'] = dataset['TransactionID'].astype(int)
dataset['TransactionID'].is_unique #True, so we use this field as index 

True

In [13]:
dataset = dataset.set_index('TransactionID')
dataset.head()
#print(dataset)

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionAmount (INR),CustomerAGE
TransactionID,,,,,,,,
1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,25.0,21
2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,27999.0,58
3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,459.0,19
4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,2060.0,42
5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,1762.5,27


## Data Visualization